In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [7]:
@tool
def multiply(a: int, b: int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a * b

In [ ]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [12]:
multiply.name, multiply.description, multiply.args

('multiply',
 'Given 2 numbers a and b this tool returns their product',
 {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}})

In [13]:
# tool binding
llm = ChatOpenAI()

In [15]:
llm_with_tools = llm.bind_tools([multiply])

In [17]:
# Tool Calling
llm_with_tools.invoke('Hi, how are you?')

AIMessage(content="Hello! I'm here and ready to help. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 60, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-27958c70-7076-413a-9f5c-795a49a32a8d-0', usage_metadata={'input_tokens': 60, 'output_tokens': 18, 'total_tokens': 78})

In [18]:
llm_with_tools.invoke('can you multiply 3 with 10?')

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AnW5ByvN7smgXhy1FdUird8P', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 63, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-949e1702-5a57-4490-b392-0b7bb827214a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_AnW5ByvN7smgXhy1FdUird8P', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 17, 'total_tokens': 80})

In [19]:
llm_with_tools.invoke('can you multiply 3 with 10?').tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 10},
  'id': 'call_zn99SUOHo1rxkXCYG9MJwZIs',
  'type': 'tool_call'}]

In [20]:
llm_with_tools.invoke('can you multiply 3 with 10?').tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_DGmMyfYjysWCMcPNRhR6j16p',
 'type': 'tool_call'}